# 2. 確率分布と統計モデルの最尤推定
表現の部品としての確率分布を理解するために、簡単なデータと確率分布の対応付けに挑戦します。

## 2.1 種子数の統計モデリング
ここでは、種子数のデータを元に集計したり、プロットをしたりします。

In [11]:
# 必要なライブラリを読み込み
require 'daru'
require 'rbplotly'
require 'daru/plotly'

true

In [32]:
include Daru::Plotly::Methods

Object

In [25]:
# データの取得（ノートブックの作詞さによって事前にCSVに変換済み）
df = Daru::DataFrame.from_csv("data/data.csv")
data_arr = df.seed_size.to_a

[2, 2, 4, 6, 4, 5, 2, 3, 1, 2, 0, 4, 3, 3, 3, 3, 4, 2, 7, 2, 4, 3, 3, 3, 4, 3, 7, 5, 3, 1, 7, 6, 4, 6, 5, 2, 4, 7, 2, 2, 6, 2, 4, 5, 4, 5, 1, 3, 2, 3]

In [4]:
# いくつのデータが含まれているか
data_arr.count

50

In [5]:
# 要約（上から：個数、平均、標準偏差、最小、最大）
df.seed_size.describe

,statistics
count,50
mean,3.56
std,1.728040060004279
min,0
max,7


In [6]:
# 度数分布を得る
df.seed_size.value_counts

2,11
4,10
6,4
5,5
3,12
1,3
0,1
7,4


In [7]:
# ヒストグラムを描く
Daru::Plotly::Methods.plot(df.seed_size.value_counts, x: 'data', y: 'Frequency', type: :bar).show

#<Plotly::Offline::HTML:0x007f96ec197f50 @id="b88b7c87-6330-41ad-9702-0f4553340b2c", @data=[{:x=>[2, 4, 6, 5, 3, 1, 0, 7], :y=>[11, 10, 4, 5, 12, 3, 1, 4], :type=>:bar, :mode=>""}], @layout={:width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<CZTop::Socket::PUB:0x7f96ee41c000 last_endpoint="tcp://127.0.0.1:59115">

In [8]:
# 分散を表す
df.seed_size.cov

0.485404511237157

In [9]:
# 標本標準偏差
df.seed_size.std

1.728040060004279

## 2.2 データと確率分布の対応関係をながめる
ここまでで種子数のデータについて以下のような特徴があることがわかりました。
* 1個, 2個・・・と数えられるデータ
* 1個体の種子数の標本平均は3.56個
* 個体ごとに種子数にばらつきがあり、ヒストグラムを描くとひと山の分布になる

## 種子数の個体データを統計モデルとして表現する
上記のような種子数データを統計モデルで表すには**ポアソン分布（Poisson distribution）**と呼ばれる確率分布が便利なため、以降ではポアソン分布を用いて統計モデルとして表現していきます。  （やや天下り的ではありますが、数学とかの勉強はこういうことが多いですよね^^）  
なお、確率分布とは簡単に言うと、**確率変数（random variable）**の値とそれが出現する確率を対応させたものです。

今回の種子数の例に当てはめて考えると、ある植物個体 $i$ の種子数 $ y_i $のように、個体ごとにばらつく変数が確率変数となります。  
ここで例えば $y_i=2$ となる場合に、そのその確率はどのくらいなのかを知るために確率分布を用いることができるのです。

In [10]:
# Rubyでは確率分布を用いるのに、 `distribution` （Sciruby）というライブラリが便利
require 'distribution'

true

In [11]:
# lambda=3.56 , y_i=2 となる確率を求める
lambda_val = 3.56
y_i = 2

# Poisson PDF for x, lambda.
Distribution::Poisson.pdf(y_i, lambda_val)

0.18021114444884437

In [12]:
ys = 0..9
lambda_val = 3.56

# Poisson PDF for x, lambda.
probs = ys.map{ |y| Distribution::Poisson.pdf(y, lambda_val)}

[0.028438824714184505, 0.10124221598249684, 0.18021114444884437, 0.21385055807929534, 0.19032699669057282, 0.1355128216436879, 0.0804042741752548, 0.04089131658055816, 0.01819663587834838, 0.007197780414102248]

In [13]:
prob_df = Daru::DataFrame.new( n_seeds: ys, probabilty: probs )
Daru::Plotly::Methods.plot(prob_df, x: :n_seeds, y: :probabilty).show

#<Plotly::Offline::HTML:0x007f96ed449838 @id="31c8a786-5560-48f5-80ab-8844254de1d6", @data=[{:x=>[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], :y=>[0.028438824714184505, 0.10124221598249684, 0.18021114444884437, 0.21385055807929534, 0.19032699669057282, 0.1355128216436879, 0.0804042741752548, 0.04089131658055816, 0.01819663587834838, 0.007197780414102248], :type=>nil, :mode=>"", :name=>:probabilty}], @layout={:width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<CZTop::Socket::PUB:0x7f96ee41c000 last_endpoint="tcp://127.0.0.1:59115">

## ばらつきのある事象・現象を記述する
統計モデリングでは上記のような確率分布を用いて、ばらつきのある事象・現象を記述できるとみなします。  
以下の図では先ほど描いた観測データのヒストグラムと上記の確率分布が似ていることが確認できます。

In [14]:
# いい感じにプロットできるようにy軸を拡大
prob_df[:probabilty_50x] = prob_df[:probabilty] * 50
Plotly::Plot.new(
  data: generate_data(df.seed_size.value_counts, x: 'data', y: 'Frequency', type: :bar) + generate_data(prob_df, x: :n_seeds, y: :probabilty_50x)
).show

#<Plotly::Offline::HTML:0x007f96eda27408 @id="4bc968fd-2e82-45e8-b3d1-c983f83ad48f", @data=[{:x=>[2, 4, 6, 5, 3, 1, 0, 7], :y=>[11, 10, 4, 5, 12, 3, 1, 4], :type=>:bar, :mode=>""}, {:x=>[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], :y=>[1.4219412357092251, 5.062110799124842, 9.010557222442218, 10.692527903964768, 9.516349834528642, 6.775641082184395, 4.02021370876274, 2.044565829027908, 0.909831793917419, 0.3598890207051124], :type=>nil, :mode=>"", :name=>:probabilty_50x}], @layout={}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<CZTop::Socket::PUB:0x7f96ee41c000 last_endpoint="tcp://127.0.0.1:59115">

## 2.3 ポアソン分布とは何か？
ここではポアソン分布とは何かについて説明していきます。  
ポアソン分布の確率分布は以下のように定義されます。

$$ p(y|\lambda)=\frac{\lambda^y exp(-\lambda)}{y!}$$

上記から、ポアソン分布では平均 $\lambda$ が唯一のパラメタで、平均 $\lambda$ が決まるとyになる確率を求めることができることが分かります。  
ポアソン分布には以下のような特徴があります。  

* $y\in{0, 1, 2, ..., \infty}$ の値をとり、 全てのyについて和をとると1になる
* 確率分布の平均は $\lambda$ になる
* 分散と平均が等しい

このポアソン分布を用いることで、ばらつきのあるデータを説明する統計モデルを作ることができます。  

## 2.4 ポアソン分布のパラメーターの最優推定
### 最優推定とは？
最優推定とは観測データに基づいて確率分布のパラメーターを推定する方法です。  
最優推定法では**尤度**という「あてはまりのよさ」を表す統計量を最大にするパラメーター（ポアソン分布だと $\lambda$ ）の値を探るパラメーター推定法です。  
尤度関数を $L(\lambda)$ として以下のように求めます。

$$L(\lambda) = \prod_{i} p(y_i|\lambda)$$

なぜ、この形になるかというと、全ての自称が同時に真となる確率を求めたいからです。  
ゆうど関数はこのままだと扱いにくいので、通常は対数変換した**対数尤度関数**を使ってパラメーターを最尤推定します。

$$log L(\lambda)  = \sum_{i} (y_i log \lambda - \lambda - \sum_{k}^{y_i} log k) $$

以下ではパラメーター $\lambda$ を変化させていたった時に、上記の対数尤度関数がどのように変化するかを調べてみます。

In [18]:
# log計算するために必要
include Math

Object

In [14]:
# 対数尤度関数
# m: lambdaの値
# data_arr: データ
def log_l(m, data_arr)
  data_arr.map{ |e| log(Distribution::Poisson.pdf(e,m))}.sum
end

:log_l

In [23]:
lambdas = (1..30).map{|i| (i * 0.1 + 2).floor(1)}

[2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0]

In [28]:
log_l_df = Daru::DataFrame.new( log_l: lambdas.map{ |e| log_l(e, data_arr)}, lambdas: lambdas )
Daru::Plotly::Methods.plot(log_l_df, x: :lambdas, y: :log_l).show

#<Plotly::Offline::HTML:0x007f96ed2df510 @id="c0aa8be0-76a9-490e-9106-ad60adbaf730", @data=[{:x=>[2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0], :y=>[-118.19653256475863, -114.9159697817477, -112.0035560441393, -109.42794467759362, -107.16162965298821, -105.18034271170413, -103.46256433075736, -101.98912366834165, -100.74286874193557, -99.70839254366427, -98.8718040811719, -98.22053578117661, -97.74318053849444, -97.42935309785122, -97.2695715344123, -97.25515543434035, -97.37813802885599, -97.63119004823126, -98.00755346845087, -98.50098364724728, -99.10569860616114, -99.81633442508839, -100.62790588607382, -101.53577164207746, -102.53560330041103, -103.62335790446902, -104.7952533751375, -106.04774653792335, -107.3775134158364, -108.78143151331794], :type=>nil, :mode=>"", :name=>:log_l}], @layout={:width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<CZTop::Socket::PUB:0x7f96ee41c000 last_endpoint="tcp://127.0.0.1:59115">

### 最優推定
上記の例では3.5〜3.6のあたり対数尤度が最大になることが分かります。  
また、対数尤度関数についてパラメータλを偏微分した式を0と置くことによって、パラメータλを求めることができます。

## 2.4.1 擬似乱数と最尤推定値のばらつき
ここでは擬似乱数発生装置を利用して、乱数列を発生させるごとに最尤推定値 $\hat{\lambda}$ がどのように変化するかを見ていきます。

In [1]:
# Rubyで特定の分布に従った擬似乱数を発生させるには「rubystats」を利用する
require 'rubystats'

true

In [3]:
# ポアソン分布の乱数発生
lambda_val = 3.56
pois_seed = Rubystats::PoissonDistribution.new(lambda_val)

#<Rubystats::PoissonDistribution:0x007f9c62aead20 @rate=3.56>

In [13]:
# Object.rng(n) で n個の乱数を生成（Radom Seed Generate）
pois_seed.rng(50)

[4, 5, 1, 3, 4, 4, 4, 3, 2, 3, 6, 3, 2, 6, 5, 1, 3, 2, 4, 2, 1, 5, 2, 1, 3, 3, 6, 5, 0, 4, 5, 3, 4, 3, 3, 2, 3, 2, 3, 3, 4, 2, 1, 3, 3, 3, 10, 2, 2, 1, 5]

In [14]:
# 最優推定をする関数
def pois_mle(ds)
  ds.inject(:+) / ds.size.to_f
end

:pois_mle

In [15]:
mls = []
(1..3000).each do
  ml = pois_mle(pois_seed.rng(50))
  mls.push(ml.floor(1))
end

1..3000

In [16]:
mls_count = Hash.new(0)
mls.each{|e| mls_count[e] += 1}
mls_count

{3.6=>431, 3.8=>249, 3.5=>462, 3.3=>337, 4.0=>96, 3.4=>407, 3.2=>235, 3.1=>136, 2.8=>7, 3.7=>326, 4.2=>19, 2.9=>26, 3.0=>77, 3.9=>148, 4.1=>29, 4.3=>7, 4.4=>4, 2.6=>1, 2.5=>1, 4.6=>1, 2.7=>1}

In [17]:
# 描画
count_df = Daru::DataFrame.new({ :num => mls_count.keys, :val => mls_count.values})
Daru::Plotly::Methods.plot(count_df, x: :num, y: :val, type: :bar).show

#<Plotly::Offline::HTML:0x007f9c632388e8 @id="bacfc0a6-ceba-4eb5-b50a-6036ec95bcf7", @data=[{:x=>[3.6, 3.8, 3.5, 3.3, 4.0, 3.4, 3.2, 3.1, 2.8, 3.7, 4.2, 2.9, 3.0, 3.9, 4.1, 4.3, 4.4, 2.6, 2.5, 4.6, 2.7], :y=>[431, 249, 462, 337, 96, 407, 235, 136, 7, 326, 19, 26, 77, 148, 29, 7, 4, 1, 1, 1, 1], :type=>:bar, :mode=>"", :name=>:val}], @layout={:width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<CZTop::Socket::PUB:0x7f9c63e8c290 last_endpoint="tcp://127.0.0.1:49527">

### ポアソン分布の平均 $\lambda$ の最優推定値 $\hat{\lambda}$ のばらつき
3000回の思考の結果、毎回50個のポアソン分布に従う乱数を生成して推定した $\hat{\lambda}$ の分布は `3.56` 付近が山のてっぺんになる

## 2.5 統計モデルの要点
### 統計モデリングの流れ
今回のポアソン分布で種子数を説明するまでの、データ解析者の頭の中は以下のような思考をたどっています。

1. `[4, 5, 1, 3, 4, 4, 4, 3, 2, 3, 6, 3, 2, 6, 5, ...]` と行った数字の羅列を見て、 **「確率分布から発生している？」** と考える
2. データのばらつきは「ポアソン分布で説明できるのでは」と考える
3. パラメーターλの値を **推定（estimation）** したい
4. 最優推定によって $\hat{\lambda} = 3.56$ が得られたので、平均3.56のポアソン分布で説明できると考える 

### 予測
統計モデルを作成したら、次は**予測**が重要になります。  
推定であられた統計モデルを使って、次にデータ分布などを見積もれます。  
予測には以下のようなものがあります。

- 次に得られる応答変数の平均を示す
- 次に得られるデータの予測区間を示す
- 時系列データで未来予測
- 空間構造のあるデータで欠損値を埋める

## 2.6 確率分布の選び方
統計分布で考えるべきことは「当該の現象がどのような確率分布で説明されそうか」ということです。  
データを見たら以下のような点に注意してください。

- 説明したい量が**離散**か**連続**か？
- 説明したい量の**範囲**は？
- 説明したい量の標本分散と標本平均の関係は？

緑本では以下のような確率分布を扱います。それぞれの分布の特徴も押さえておくようにしましょう。
- 離散
  - ポアソン分布
  - 二項分布
- 連続
  - 正規分布
  - ガンマ分布
  - 一様分布
 